### TF-KOMPAS: Site caller

Updated 11/27/19

Author: Zachery Mielko

Contributors and Collaborators:
* Tiffany Ho: optimization review, feedback, and validation
* Farica Zhuang: review, feedback, and validation

-----

The script requires the following dependencies:

* pandas
* numpy
* biopython
* pybedtools

The script gives the following as **output**:
- Bed file of centered sites (Centered_KOMPAS.bed)
    - The Bed file is 0 based and of the format start inclusive, end exclusive. So half-open.
    - Columns: Chromosome, Start, End, Orientation, Threshold Score
- log file (optional)

By default, the center position will be the midpoint of the core (rounded down), but it can be specifically chosen as an optional parameter.


**Threshold Score**

This is the threshold for when the site would be called. 

If k > core length (if multiple kmers exist that can fully describe the core), then the second score from **maximum** for those kmers is used. This is due to the requirnment of 2 overlapping kmers.

If k <= core length (if multiple (or 1) kmers are needed to describe the core fully), then the **minimum** score for those kmers is used because only when threshold is set to that minimum or below would the site be called

**Handles both palindromes and non-palindromes.**

If the query is for a palindrome, there is an option isPalindome to set to True. All it does is remove - strand calls, since any match on one strand is a match on another. If this is set to false for a palindrome, it should give + and - matches representing both centered positions, or 2 calls per binding site.

-------
Latest update (11/27/19) - ZM
* Completely redone site calling implementation using numpy arrays.
    - Fixes an issue with overlapping cores, especially at low Escore thresholds. 
    - Now KOMPAS can call overlapping cores in the same manner at any threshold. 
    - Removes a couple dependencies, though they are packeged with python by default
* Fixed a bug with the threshold scoring
* Optimized palindrome mode, should run 2x as fast as non-palindrome mode

Update (11/25/19) - ZM
* Diagnostic tsv file shows all searched peaks, even if no calls were made
* Threshold score is now set up so you can use it as a substitute for running at different thresholds. Represents practical threshold given 2 overlapping, not theoretical. 


Update (11/20/19) - ZM
* Fixed bug in coordinates for 0 vs 1 based. KOMPAS is now fully in-sync with half-open, 0 based formats used by bedtools, MACS, and UCSC (backend)


In [4]:
################## Parameters ###############
# Files and folders
wkdir = "/Users/ZMielko/Desktop/Test_KOMPAS/"
peakFile = '/Users/ZMielko/Desktop/Test_KOMPAS/ets1_k562_ENCSR000BKQ_idr.txt'
genomeFile = '/Users/ZMielko/Desktop/In_Vivo_Project/Data/human_g1k_v37.fasta'
kmerFile = '/Users/ZMielko/Desktop/Test_KOMPAS/aligned_ets1_8mer.txt'
# kPosition of the core and where to center the call
# core is right exclusionary, left inclusive [)
core = (10,16) 
threshold = 0.3
isPalindrome = False
# Optional settings
logFile = True
centerPos = 12 # if set to 'default', will be half position of the core rounded down

#############################################
if centerPos == 'default':
    centerPos = int((core[0] + core[1])/2)
##### Imports ####

import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import reverse_complement
import os
from pybedtools import BedTool

#### Functions ####
def readFasta(file):
    '''
    Reads a fasta file and turns it into a dataframe with forward and rc sequences
    '''
    entry = []
    with open(file, "r") as input_handle:
        for record in SeqIO.parse(input_handle, "fasta"):
            entry.append([record.id, str(record.seq).upper(), len(record.seq), str(reverse_complement(record.seq)).upper()])
    arr = np.array(entry)
    df = pd.DataFrame({'seq_id':arr[:, 0], 'fwd':arr[:,1], 'seq_len':arr[:,2], 'rev_comp':arr[:,3]})
    return df
  
def parseID(df):
    """parseID takes the concatinated names given in fasta outputs from bedtool's getfasta 
    and turns them into bed compatible columns"""
    chrom, start, end = [], [], []
    for i in df.seq_id:
        cr = i.split(':')
        pos = cr[1].split('-')
        chrom.append(cr[0])
        start.append(int(pos[0]))
        end.append(int(pos[1]))
    df['Chromosome'] = chrom
    df['Start'] = start
    df['End'] = end
    return(df)

def convertToBed(df, isPalindrome):
    if isPalindrome == False:
        chrom, start, orient,scores = [],[],[],[]
        for row in zip(df['Chromosome'],df['Start'],df['centerPlus'],df['scorePlus'],df['centerMinus'],df['scoreMinus']):
            if row[2]:
                for centerP, score in zip(row[2], row[3]): # + sites
                    chrom.append(row[0])
                    start.append(row[1] + centerP)
                    orient.append('+')
                    scores.append(score)
            if row[4]:
                for centerN, score in zip(row[4],row[5]): # - sites
                    chrom.append(row[0])
                    start.append(row[1] + centerN)
                    orient.append('-')
                    scores.append(score)
    else:
        for row in zip(df['Chromosome'],df['Start'],df['centerPlus'],df['scorePlus']):
            if row[2]:
                for centerP, score in zip(row[2], row[3]): # + sites
                    chrom.append(row[0])
                    start.append(row[1] + centerP)
                    orient.append('+')
                    scores.append(score)
    bedDF = pd.DataFrame({'chrom':chrom,'start':start, 'end':start,'orient':orient, 'score':scores})
    bedDF['end'] = bedDF['end'] + 1 # exclusive end position
    bedDF = bedDF.drop_duplicates().sort_values(by=['chrom','start']) # some sites overlap, will call the same centers
    return(bedDF)
##### Read in kmer data and process ####
kmer = pd.read_csv(kmerFile, sep = '\t')
k = len(kmer['kmer'][0])
coreLen = core[1] - core[0]
# Find the kPositions required, any would be sufficient to call
if k > coreLen: 
    searchEnd = core[1]
    checkK = 0
    ReqKpos = set() #
    while checkK != core[0]:
        checkK = searchEnd - k
        if checkK <= core[0]:
            ReqKpos.add(checkK)
            searchEnd = searchEnd + 1
# Or find the group of all kPositions that are needed, all or none
else:
    searchStart = core[0]
    checkK = 0
    ReqKpos = set()
    while searchStart + k <= core[1]:
        ReqKpos.add(searchStart)
        searchStart = searchStart + 1
# Determine flanks of ReqKPos for threshold score reporting
ScoredKpos = ReqKpos.copy()
if k >= coreLen:
    ScoredKpos.add(min(ReqKpos) - 1)
    ScoredKpos.add(max(ReqKpos) + 1)        

# Generate dictionary for quick retreaval of relevant kmers
thrKmers = kmer[(kmer['Escore'] > threshold) & (kmer['kposition'].isin(ScoredKpos))]
kDict = dict(zip(thrKmers['kmer'],zip(thrKmers['kposition'],thrKmers['Escore'])))


##### Generate a fasta file from peakFile ####
peaks = pd.read_csv(peakFile, sep = '\t',header = None, usecols=[0,1,2])
# find out how long the peaks need to be to use them
if coreLen > k:
    minLen = coreLen
else:
    minLen = k + 1
peaks = peaks[peaks[2]-peaks[1] > (minLen)].drop_duplicates() # filter for short sequences the caller would have trouble with
# Generate peaks using bedtools
peakBed = BedTool.from_dataframe(peaks)
peakSequence = peakBed.sequence(fi = genomeFile)
peakDF = readFasta(peakSequence.seqfn)
peakDF = parseID(peakDF)
del peakDF['seq_id']
peakDF[['seq_len', 'Start','End']] = peakDF[['seq_len', 'Start','End']].astype(int) 

def kmerMatch(seq):
    """
    Returns matched positions in the sequence and their kpositions
    Input: sequence and kmer length (k)
    Output: consecutive positions, kpositions, and scores above threshold
    """
    # Get the kposition and kscore for the peak, save a numpy array
    kpos,kscore = [], []
    for i in range(len(seq) - k + 1):
        window = seq[i:i+k]
        if window in kDict:
            kpos.append(kDict[window][0])
            kscore.append(kDict[window][1])
        else:
            kpos.append(0)
            kscore.append(-0.5)
    kpos = np.array(kpos)
    kscore = np.array(kscore)
    # Get consecutive positions, kpositions, and score via numpy operations
    if k >= coreLen:
        position = list(filter(lambda x: len(x) != 1,np.split(np.r_[:len(kpos)], np.where(np.diff(kpos) != 1)[0]+1)))
        kpos = list(filter(lambda x: len(x) != 1,np.split(kpos, np.where(np.diff(kpos) != 1)[0]+1)))
    elif k < coreLen:
        reqLen = len(ReqKpos)
        position = list(filter(lambda x: len(x) == reqLen,np.split(np.r_[:len(kpos)], np.where(np.diff(kpos) != 1)[0]+1)))
        kpos = list(filter(lambda x: len(x) == reqLen,np.split(kpos, np.where(np.diff(kpos) != 1)[0]+1)))
    kScore = []
    for pos in position:
        kScore.append(kscore[pos])
    return(zip(position, kpos, kScore))

def findCenter(zippedMatch, orient, seqLen):
    """
    Given a zip of match position, kposition, and kscore
    Returns the center sites and threshold kscore
    """
    centerSites = []
    siteScores = []
    for pos, kpos, kScore in zippedMatch:
        centerSite = (centerPos - kpos[0]) + pos[0]
        if orient == 'rc':
            centerSite = (seqLen - centerSite) -1
        centerSites.append(centerSite)
        if k >= coreLen:
            score = threshold
            for score1, score2 in zip(kScore, kScore[1:]):
                caniScore = sorted([score1, score2])[0]
                if caniScore > score:
                    score = caniScore
            siteScores.append(score)
        elif k < coreLen:
            siteScores.append(min(kScore))
    return(pd.Series([centerSites, siteScores]))
             
# Run kmerMatch on the peaks
if isPalindrome == True:
    peakDF[["centerPlus","scorePlus"]] = peakDF.apply(lambda peakDF: findCenter(kmerMatch(peakDF['fwd']), 'fwd', len(peakDF['fwd'])), axis = 1) 
    peakDF = peakDF[~peakDF['centerPlus'].isna()]
else:
    peakDF[["centerPlus","scorePlus"]] = peakDF.apply(lambda peakDF: findCenter(kmerMatch(peakDF['fwd']), 'fwd',len(peakDF['fwd'])), axis = 1) 
    peakDF[["centerMinus","scoreMinus"]] = peakDF.apply(lambda peakDF: findCenter(kmerMatch(peakDF['rev_comp']), 'rc',len(peakDF['rev_comp'])), axis = 1)
    peakDF = peakDF[~peakDF['centerPlus'].isna() | ~peakDF['centerMinus'].isna()]

finalBed = convertToBed(peakDF, isPalindrome)
finalBed.to_csv(f'{wkdir}Centered_{threshold}_KOMPAS.bed', sep = '\t', header = None, index = False)

if logFile == True:
    ##################
    # Log the output #
    ##################
    f = open(wkdir + "/KOMPASLog.txt", "a")
    f.write("##### Parameters ##### \n")
    f.write(f"Peak file: {peakFile}"+ "\n")
    f.write(f"Genome file: {genomeFile}"+ "\n")
    f.write(f"kmer file: {kmerFile}"+ "\n")
    f.write(f"Core kPositions: {core}"+ "\n")
    f.write(f"Center kPositions: {centerPos}"+ "\n")
    f.write(f"Threshold: {threshold}"+ "\n")
    f.write("#Summary# \n")
    f.write(f"Final # of called sequences: {len(finalBed)}"+ "\n")
    f.close()
